In [1]:
#import stuff
import os
import sys
import arrow
import numpy as np
import pandas as pd
from math import pi
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import seaborn as sns
from datetime import datetime
from numpy import nan_to_num
from matplotlib.colors import LogNorm
from matplotlib.ticker import ScalarFormatter
import icartt

In [2]:
filename = 'C:/Users/cphal/Desktop/MAC prediction model/FIREXAQ-LARGE-CDP_DC8_20190722_R0.ict'

In [3]:
#open and read file
f = open(filename)
nextline = f.readline()

#FIXME: not sure if this will be true for all files
#extract and ignore the data we don't care about
while '-9999' not in nextline:
    nextline = f.readline()

#create list to store description of variables (to extract bin sizes from)
bin_size_description = []
nextline = f.readline()
while len(nextline) > 10:
    nextrow = nextline.split(',')
    bin_size_description.append(nextrow[3])
    nextline = f.readline()

#delete the first two ones
del bin_size_description[0]
del bin_size_description[0]
#FIXME: this will not be true for all files

#create new list with just the numbers
bin_sizes = []
for x in range(len(bin_size_description)):
    #extract beginning of description
    starting_string = 'number_size_distribution_dNdlogD_at_bin_center_'
    string_length = len(starting_string) + 1
    
    #build number and add to list
    str1 = bin_size_description[x]
    c = str1[string_length]
    new_bin_size = ''
    while(c != 'u'):
        new_bin_size = new_bin_size + c
        string_length += 1
        c = str1[string_length]
    bin_sizes.append(float(new_bin_size))

size_dist_diameter_input = pd.DataFrame(bin_sizes)
size_dist_diameter_input.columns = ['diameter']
    
f.close()

In [78]:
type(size_dist_diameter_input)

pandas.core.frame.DataFrame

In [4]:
#purpose: import icartt data and convert to dataframe

#set directory
#fileDirectory = str('c:/Users/cphal/Desktop/NOAA_DATA/')
#prompt user to enter campaign name
#campaign = input('Campaign Name: ')
#campaign = 'firexaq'
#campaign = campaign.upper()

#get filenames in folder
#path = os.chdir(str(fileDirectory + campaign + '/'))
#folder_files = []
#with os.scandir(path) as entries:
    #for entry in entries:
        #folder_files.append(entry.name)

filename = 'C:/Users/cphal/Desktop/MAC prediction model/FIREXAQ-LARGE-CDP_DC8_20190722_R0.ict'

#load in ict file to python
ict = icartt.Dataset(filename)
df = ict.data[:]
varnames = [x for x in ict.variables]
df = pd.DataFrame(df, columns = varnames)

In [5]:
#put first column into datetime file; next two columns are nCDP and vCDP
#last columns are the bins - size_dist_input

#time after midnight from dataframe - may not be needed
#datetime = df['Time_mid']
totalCDP = df[['nCDP','vCDP']]

binnames = varnames[3:len(varnames)]
size_dist_input = df[binnames]
print(size_dist_input)


       CDP_Bin00  CDP_Bin01  CDP_Bin02  CDP_Bin03  CDP_Bin04  CDP_Bin05  \
0            NaN        NaN        NaN        NaN        NaN        NaN   
1            NaN        NaN        NaN        NaN        NaN        NaN   
2            NaN        NaN        NaN        NaN        NaN        NaN   
3            NaN        NaN        NaN        NaN        NaN        NaN   
4            NaN        NaN        NaN        NaN        NaN        NaN   
...          ...        ...        ...        ...        ...        ...   
24741   0.043059        0.0   0.000000   0.000000   0.000000        0.0   
24742   0.000000        0.0   0.000000   0.000000   0.000000        0.0   
24743   0.000000        0.0   0.454437   0.556186   0.000000        0.0   
24744   0.045596        0.0   0.000000   0.000000   0.000000        0.0   
24745   0.000000        0.0   0.492248   0.000000   0.712563        0.0   

       CDP_Bin06  CDP_Bin07  CDP_Bin08  CDP_Bin09  ...  CDP_Bin20  CDP_Bin21  \
0            NaN   

In [6]:
#Implement Module A (w/o merge) (these are the dividing lines)
d_N1 = int(50)
d_N2 = int(200)
d_V1 = int(1000)
d_V2 = int(2500)

In [7]:
date_time = ict.times
datetimedf = pd.DataFrame(date_time, columns= ['datetime'])
print(date_time)
num1 = str(date_time[0])
print(type(num1))
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000000000')

['2019-07-22T18:15:40.000000000' '2019-07-22T18:15:41.000000000'
 '2019-07-22T18:15:42.000000000' ... '2019-07-23T01:08:03.000000000'
 '2019-07-23T01:08:04.000000000' '2019-07-23T01:08:05.000000000']
<class 'str'>


In [8]:
#FIX: improve efficiency
size_dist_volume = pd.DataFrame().reindex_like(size_dist_input)
for m in range(np.size(size_dist_volume,0)):
    for n in range(len(size_dist_diameter_input)):
        #calculation for aerosol volume distribution - based on aerosol size distribution
        size_dist_volume.values[m,n]=pi/6* np.power(size_dist_diameter_input, 3).values[n]*size_dist_input.values[m,n]

In [73]:
#print(size_dist_volume.values[1800,1])
print(size_dist_volume)

       CDP_Bin00  CDP_Bin01  CDP_Bin02  CDP_Bin03   CDP_Bin04  CDP_Bin05  \
0            NaN        NaN        NaN        NaN         NaN        NaN   
1            NaN        NaN        NaN        NaN         NaN        NaN   
2            NaN        NaN        NaN        NaN         NaN        NaN   
3            NaN        NaN        NaN        NaN         NaN        NaN   
4            NaN        NaN        NaN        NaN         NaN        NaN   
...          ...        ...        ...        ...         ...        ...   
24741   0.352279        0.0   0.000000    0.00000    0.000000        0.0   
24742   0.000000        0.0   0.000000    0.00000    0.000000        0.0   
24743   0.000000        0.0  21.682515   48.45142    0.000000        0.0   
24744   0.373032        0.0   0.000000    0.00000    0.000000        0.0   
24745   0.000000        0.0  23.486603    0.00000  102.461738        0.0   

       CDP_Bin06  CDP_Bin07  CDP_Bin08  CDP_Bin09  ...  CDP_Bin20  CDP_Bin21  \
0      

In [19]:
N1 = np.zeros(np.size(size_dist_input,0)); N2 = np.zeros(np.size(size_dist_input,0)); N3 = np.zeros(np.size(size_dist_input,0))
F_N1 = np.zeros(np.size(size_dist_input,0)); F_N2 = np.zeros(np.size(size_dist_input,0)); F_N3 = np.zeros(np.size(size_dist_input,0))   
areaXY_number = np.zeros(np.size(size_dist_input,0))  
V1 = np.zeros(np.size(size_dist_input,0)); V2 = np.zeros(np.size(size_dist_input,0)); V3 = np.zeros(np.size(size_dist_input,0))
F_V1 = np.zeros(np.size(size_dist_input,0)); F_V2 = np.zeros(np.size(size_dist_input,0)); F_V3 = np.zeros(np.size(size_dist_input,0))   
areaXY_volume = np.zeros(np.size(size_dist_input,0))

# prepare for trapz calculation
size_dist_input[np.isnan(size_dist_input)] = 0 
size_dist_volume[np.isnan(size_dist_volume)] = 0 

ln_size_dist_diameter_input=np.log(size_dist_diameter_input)
print(ln_size_dist_diameter_input)

    diameter
0   0.916291
1   1.252763
2   1.504077
3   1.704748
4   1.871802
5   2.014903
6   2.140066
7   2.251292
8   2.351375
9   2.442347
10  2.525729
11  2.602690
12  2.708050
13  2.833213
14  2.944439
15  3.044522
16  3.135494
17  3.218876
18  3.295837
19  3.367296
20  3.433987
21  3.496508
22  3.555348
23  3.610918
24  3.663562
25  3.713572
26  3.761200
27  3.806662
28  3.850148
29  3.891820


C:\Users\cphal\AppData\Local\Temp\ipykernel_16236\1251108476.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size_dist_input[np.isnan(size_dist_input)] = 0
C:\Users\cphal\AppData\Local\Temp\ipykernel_16236\1251108476.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size_dist_input[np.isnan(size_dist_input)] = 0


In [16]:
print(size_dist_input)

       CDP_Bin00  CDP_Bin01  CDP_Bin02  CDP_Bin03  CDP_Bin04  CDP_Bin05  \
0       0.000000        0.0   0.000000   0.000000   0.000000        0.0   
1       0.000000        0.0   0.000000   0.000000   0.000000        0.0   
2       0.000000        0.0   0.000000   0.000000   0.000000        0.0   
3       0.000000        0.0   0.000000   0.000000   0.000000        0.0   
4       0.000000        0.0   0.000000   0.000000   0.000000        0.0   
...          ...        ...        ...        ...        ...        ...   
24741   0.043059        0.0   0.000000   0.000000   0.000000        0.0   
24742   0.000000        0.0   0.000000   0.000000   0.000000        0.0   
24743   0.000000        0.0   0.454437   0.556186   0.000000        0.0   
24744   0.045596        0.0   0.000000   0.000000   0.000000        0.0   
24745   0.000000        0.0   0.492248   0.000000   0.712563        0.0   

       CDP_Bin06  CDP_Bin07  CDP_Bin08  CDP_Bin09  ...  CDP_Bin20  CDP_Bin21  \
0            0.0   

In [17]:
 #function purpose: find nearest value (d_N1, d_N2, d_V1, d_V2) in numpy array (instrument measured diameters)
def find_nearest(array, value):
    n = [abs(i-value) for i in array]
    idx = n.index(min(n))
    return idx

In [21]:
print(shape(size_dist_diameter_input))
print(shape(size_dist_input.values))
m=0
#areaXY_number[m] = np.trapz(size_dist_input.values[:,m], x=ln_size_dist_diameter_input)

NameError: name 'shape' is not defined

In [18]:
for m in range(np.size(size_dist_input,0)):

        d_N1_nearest = find_nearest(size_dist_diameter_input['diameter'],d_N1)
        d_N2_nearest = find_nearest(size_dist_diameter_input['diameter'],d_N2)
        d_V1_nearest = find_nearest(size_dist_diameter_input['diameter'],d_V1)
        d_V2_nearest = find_nearest(size_dist_diameter_input['diameter'],d_V2)
        areaXY_number[m] = np.trapz(size_dist_input.values[m,0:size_dist_input], x=ln_size_dist_diameter_input)
        N1[m] = np.trapz(size_dist_input.values[m,0:d_N1_nearest], x=ln_size_dist_diameter_input[0:d_N1_nearest])
        N2[m] = np.trapz(size_dist_input.values[m,d_N1_nearest+1:d_N2_nearest], x=ln_size_dist_diameter_input[d_N1_nearest+1:d_N2_nearest])    
        N3[m] = np.trapz(size_dist_input.values[m,d_N2_nearest+1:], x=ln_size_dist_diameter_input[d_N2_nearest+1:])
        F_N1[m] = N1[m]/areaXY_number[m]
        F_N2[m] = N2[m]/areaXY_number[m]       
        F_N3[m] = N3[m]/areaXY_number[m]

        areaXY_volume[m] = np.trapz(size_dist_volume.values[m,:], x=ln_size_dist_diameter_input)
        V1[m] = np.trapz(size_dist_volume.values[m,0:d_V1_nearest], x=ln_size_dist_diameter_input[0:d_V1_nearest])
        V2[m] = np.trapz(size_dist_volume.values[m,d_V1_nearest+1:d_V2_nearest], x=ln_size_dist_diameter_input[d_V1_nearest+1:d_V2_nearest])    
        V3[m] = np.trapz(size_dist_volume.values[m,d_V2_nearest+1:], x=ln_size_dist_diameter_input[d_V2_nearest+1:])
        F_V1[m] = V1[m]/areaXY_volume[m]
        F_V2[m] = V2[m]/areaXY_volume[m]       
        F_V3[m] = V3[m]/areaXY_volume[m]

ValueError: operands could not be broadcast together with shapes (30,0) (29,) 

In [92]:
# If the selected size classes are broader than the measured aerosol sizes 
print (d_N1,d_N2,d_V1,d_V2,size_dist_diameter_input[len(size_dist_diameter_input)-1])
if (d_N1 < size_dist_diameter_input[0]):  #only large particles is available
    N1=0; F_N1=0  
if (d_N2 > size_dist_diameter_input[len(size_dist_diameter_input)-1]):  #only small particles is available
    N3=0; F_N3=0
if (d_V1 < size_dist_diameter_input[0]):
    V1=0; F_V1=0
if (d_V2 > size_dist_diameter_input[len(size_dist_diameter_input)-1]):
    V3=0; F_V=0

delta_days= arrow.get(datetime.dt.date[len(datetime.dt.date)-1])-arrow.get(datetime.dt.date[0])
delta_days=delta_days.days+1
delta_hourly=delta_days*24

KeyError: 29

In [13]:
#calculate hourly averaged concentration
N1_hourly=np.zeros(delta_hourly); N2_hourly=np.zeros(delta_hourly); N3_hourly=np.zeros(delta_hourly)
areaXY_number_hourly=np.zeros(delta_hourly)
F_N1_hourly=np.zeros(delta_hourly); F_N2_hourly=np.zeros(delta_hourly); F_N3_hourly=np.zeros(delta_hourly)
V1_hourly=np.zeros(delta_hourly); V2_hourly=np.zeros(delta_hourly); V3_hourly=np.zeros(delta_hourly)
areaXY_volume_hourly=np.zeros(delta_hourly)
F_V1_hourly=np.zeros(delta_hourly); F_V2_hourly=np.zeros(delta_hourly); F_V3_hourly=np.zeros(delta_hourly)
datetime_hourly=np.zeros(delta_hourly, dtype='datetime64[s]')

In [15]:
m=0
for i  in range(delta_hourly):
        areaXY_number_hourly[m] = np.nanmean(areaXY_number[i*12:(i+1)*12])  
        N1_hourly [m] = np.nanmean(N1[i*12:(i+1)*12])  
        N2_hourly [m] = np.nanmean(N2[i*12:(i+1)*12])  
        N3_hourly [m] = areaXY_number_hourly[m]- N1_hourly [m] -N2_hourly [m] #make sure N1_hourly + N2_hourly + N3_hourly = areaXY_hourly
        F_N1_hourly [m] = np.nanmean(F_N1[i*12:(i+1)*12])  
        F_N2_hourly [m] = np.nanmean(F_N2[i*12:(i+1)*12])  
        F_N3_hourly [m] = 1-F_N1_hourly [m]-F_N2_hourly [m] #make sure F_N1_hourly + F_N2_hourly + F_N3_hourly = 1
        areaXY_volume_hourly[m] = np.nanmean(areaXY_volume[i*12:(i+1)*12])  
        V1_hourly [m] = np.nanmean(V1[i*12:(i+1)*12])  
        V2_hourly [m] = np.nanmean(V2[i*12:(i+1)*12])  
        V3_hourly [m] = areaXY_volume_hourly[m]- V1_hourly [m] -V2_hourly [m] #make sure N1_hourly + N2_hourly + N3_hourly = areaXY_hourly
        F_V1_hourly [m] = np.nanmean(F_V1[i*12:(i+1)*12])  
        F_V2_hourly [m] = np.nanmean(F_V2[i*12:(i+1)*12])  
        F_V3_hourly [m] = 1-F_V1_hourly [m]-F_V2_hourly [m] #make sure F_N1_hourly + F_N2_hourly + F_N3_hourly = 1
        datetime_hourly[m] = datetime[i*12]  
        m+=1

In [16]:
file_M_1 = pd.DataFrame() 
file_M_1['datetime'] = datetime
file_M_1['N1'] = N1; file_M_1['N2'] = N2 ; file_M_1['N3'] = N3 ; file_M_1['N_total'] = areaXY_number 
file_M_1['F_N1'] = F_N1; file_M_1['F_N2'] = F_N2 ; file_M_1['F_N3'] = F_N3 
file_M_1['V1'] = V1; file_M_1['V2'] = V2 ; file_M_1['V3'] = V3 ; file_M_1['V_total'] = areaXY_volume 
file_M_1['F_V1'] = F_V1; file_M_1['F_V2'] = F_V2 ; file_M_1['F_V3'] = F_V3 

#return dataframe
print(file_M_1)

                datetime           N1           N2         N3      N_total  \
0    2018-12-01 00:00:00  2746.029133  1107.904348  50.989841  4019.385935   
1    2018-12-01 00:05:00  2499.435932  1105.918893  55.886110  3764.684852   
2    2018-12-01 00:10:00  2340.110175  1089.361886  50.916489  3597.645730   
3    2018-12-01 00:15:00  2256.101336  1160.807634  55.187856  3586.282183   
4    2018-12-01 00:20:00  2204.724486  1166.117307  56.974034  3542.060087   
...                  ...          ...          ...        ...          ...   
5755 2018-12-20 23:35:00  1419.490295   569.819421  39.613712  2099.880912   
5756 2018-12-20 23:40:00  1337.899698   576.126252  39.759802  2032.720493   
5757 2018-12-20 23:45:00  1348.586619   568.204541  42.250234  2037.099283   
5758 2018-12-20 23:50:00  1390.750324   601.209443  43.791553  2116.206785   
5759 2018-12-20 23:55:00  1364.234034   547.469573  32.726146  2019.360917   

          F_N1      F_N2      F_N3            V1            V2 